In [1]:
nome = "Mateus Oliveira"
print(f'Meu nome é {nome}')

Meu nome é Mateus Oliveira


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 6.6 MB 53.9 MB/s 
     |████████████████████████████████| 596 kB 63.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 22:20:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.examples = []
        for text in tqdm_notebook(texts):
            token_ids = tokenize(text=text, tokenizer=tokenizer)
            if len(token_ids) < context_size + 1:
                continue
            # Compute n-grams:

            # larissa trick
            for i in range(len(token_ids) - context_size):
                # input_ids = 
                # target_id = 
                self.examples.append((token_ids[i:i + context_size], token_ids[i + context_size])) 

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        input_ids, target_id = self.examples[idx]
        return torch.LongTensor(input_ids), target_id

## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-25 22:20:22--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.212.128, 172.217.204.128, 142.250.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   161MB/s    in 0.7s    

2022-05-25 22:20:23 (161 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [10]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

print('Truncating for debugging purposes.')
#texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

Truncating for debugging purposes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/24800 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
#torch.save(model, "gdrive/MyDrive/Colab Notebooks/"+"model03.pt")
#!ls "gdrive/MyDrive/Colab Notebooks"

In [13]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675945
valid examples: 82070
test examples: 166726


In [14]:
from torch.autograd import Variable

class PositionalEncoder(nn.Module):
    def __init__(self, embedding_dim, max_seq_len = 80):
        super().__init__()
        self.embedding_dim = embedding_dim
        
        pe = torch.zeros(max_seq_len, self.embedding_dim)
        for pos in range(max_seq_len):
            for i in range(0, embedding_dim, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/embedding_dim)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/embedding_dim)))
                
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger
        
        x = x * math.sqrt(self.embedding_dim)
        #add constant to embedding
        seq_len = x.size(1)
        #print(f'pos encoder {x.shape}  {self.pe[:,:seq_len].mean(axis=1).shape}')

        #raise('organize')

        x = x + Variable(self.pe[:,:seq_len], \
        requires_grad=False)# AVALIABLE ON GPU

        
        return x

In [15]:
import torch.nn.functional as F

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):

        super().__init__()
        
        self.context_size = context_size
        self.embeddings_dim = embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.posicional_encoder = PositionalEncoder(embedding_dim)

        self.W_q = nn.Linear(embedding_dim, embedding_dim, bias=False) 
        self.W_k = nn.Linear(embedding_dim, embedding_dim, bias=False) 
        self.W_v = nn.Linear(embedding_dim, embedding_dim, bias=False) 
        self.W_o = nn.Linear(embedding_dim, embedding_dim, bias=False)

        hidden_size = 2*embedding_dim

        self.layer1 = nn.Linear(self.context_size*self.embeddings_dim, hidden_size)

        self.layer2 = nn.Linear(hidden_size, 2*hidden_size)
        
        self.bn1 = nn.BatchNorm1d(2*hidden_size, affine=False)

        self.layer3 = nn.Linear(2*hidden_size,vocab_size)

        

    def Attention(self, q, k, v):

        scores = torch.matmul(q, k.transpose(2,1))# shape = B,L,L

        probs =  F.softmax(scores, dim=-1) # B,L,L
        E  = torch.matmul(probs, v) # shape = B,L,E

        return E

    def forward(self, inputs,debug=False):

        X_emb = self.embeddings(inputs)

        self.posicional_encoder(X_emb)

        residual = X_emb

        q = self.W_q(X_emb).to(device) #Shape = (B,L,D)
        k = self.W_k(X_emb).to(device) #Shape = (B,L,D)
        v = self.W_v(X_emb).to(device) #Shape = (B,L,D)

        X_emb = self.Attention(q=q, k=k, v=v)

        X_emb = self.W_o(X_emb)

        X_emb += residual
        
        hidden = torch.relu(self.layer1(X_emb.reshape(len(inputs),-1)))
        logits = torch.relu(self.layer2(hidden))

        logits = self.bn1(logits)

        logits = self.layer3(logits)


        return logits

In [16]:
# # With Learnable Parameters
# m = nn.BatchNorm1d(100)
# # Without Learnable Parameters
# m = nn.BatchNorm1d(100, affine=False)
# input = torch.randn(2, 100)
# input.shape
# output = m(input)

## Teste o modelo com um exemplo

In [17]:
model = 0

In [18]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64,
).to(device)
# # model = Transformer(
# #     vocab_size=tokenizer.vocab_size,
# #     context_size=context_size,
# #     embedding_dim=64,
# # ).to(device)

# sample_train, _ = next(iter(DataLoader(training_dataset)))
# sample_train_gpu = sample_train.to(device)
# model(sample_train_gpu).shape

In [19]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 9687138


## Assert da Perplexidade


In [20]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)

logits = model(sample_train_gpu)
print(target_token_ids.shape)
print(logits.shape)
my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

ValueError: ignored

## Laço de Treinamento e Validação

In [45]:
class SaveBestModel:

    def __init__(
        self, best_valid_loss=float('inf')
    ):
        self.best_valid_loss = best_valid_loss
        
    def __call__(
        self, current_valid_loss, 
        epoch, model, optimizer, criterion
    ):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
            print(f"Best validation loss: {self.best_valid_loss}")
            torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, "gdrive/MyDrive/Colab Notebooks/"+"best_model_26_may.pt")


In [27]:
max_examples = 1000_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)


load_dict = torch.load("gdrive/MyDrive/Colab Notebooks/"+"best_model_25_may.pt")
model.load_state_dict(load_dict['model_state_dict'])
model.to(device)


train_loader = DataLoader(training_dataset, batch_size=2048, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=2048)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()

save_best_model = SaveBestModel()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))
            
            last_loss = valid_ppl

            save_best_model(
              last_loss, 0, model, optimizer, nn.functional.cross_entropy
              )

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

Best validation loss: 175.96022504154064
0 steps; 0 examples so far; train ppl: 152.82, valid ppl: 175.96
Best validation loss: 116.83411232884983
10000 steps; 20480000 examples so far; train ppl: 99.56, valid ppl: 116.83
Best validation loss: 115.37273504902046
20000 steps; 40960000 examples so far; train ppl: 91.55, valid ppl: 115.37
Best validation loss: 115.05466868165766
30000 steps; 61440000 examples so far; train ppl: 89.64, valid ppl: 115.05
Best validation loss: 113.53289172430604
40000 steps; 81920000 examples so far; train ppl: 88.46, valid ppl: 113.53
50000 steps; 102400000 examples so far; train ppl: 86.58, valid ppl: 113.61
60000 steps; 122880000 examples so far; train ppl: 86.06, valid ppl: 114.26
70000 steps; 143360000 examples so far; train ppl: 85.79, valid ppl: 114.12
80000 steps; 163840000 examples so far; train ppl: 85.52, valid ppl: 113.56
90000 steps; 184320000 examples so far; train ppl: 84.32, valid ppl: 113.70
100000 steps; 204800000 examples so far; train ppl

KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [23]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()

load_dict = torch.load("gdrive/MyDrive/Colab Notebooks/"+"best_model_19_may.pt")
model.load_state_dict(load_dict['model_state_dict'])
model.to(device)

test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 118.13263642026706


In [24]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()

load_dict = torch.load("gdrive/MyDrive/Colab Notebooks/"+"best_model_23_may.pt")
model.load_state_dict(load_dict['model_state_dict'])
model.to(device)

test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 112.4322993216332


In [25]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()

load_dict = torch.load("gdrive/MyDrive/Colab Notebooks/"+"best_model_25_may.pt")
model.load_state_dict(load_dict['model_state_dict'])
model.to(device)

test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 112.09751392739932


In [28]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()

load_dict = torch.load("gdrive/MyDrive/Colab Notebooks/"+"best_model_25_may.pt")
model.load_state_dict(load_dict['model_state_dict'])
model.to(device)

test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 106.2128006539741


In [46]:
import torch.nn.functional as F

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):

        super().__init__()
        
        self.context_size = context_size
        self.embeddings_dim = embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.posicional_encoder = PositionalEncoder(embedding_dim)

        self.W_q = nn.Linear(embedding_dim, embedding_dim, bias=False) 
        self.W_k = nn.Linear(embedding_dim, embedding_dim, bias=False) 
        self.W_v = nn.Linear(embedding_dim, embedding_dim, bias=False) 
        self.W_o = nn.Linear(embedding_dim, embedding_dim, bias=False)

        hidden_size = 2*embedding_dim

        self.layer1 = nn.Linear(self.context_size*self.embeddings_dim, hidden_size)

        self.layer2 = nn.Linear(hidden_size, 2*hidden_size)
        
        self.bn1 = nn.BatchNorm1d(2*hidden_size, affine=False)

        self.dropout = nn.Dropout(0.2)

        self.layer3 = nn.Linear(2*hidden_size,vocab_size)

        

    def Attention(self, q, k, v):

        scores = torch.matmul(q, k.transpose(2,1))# shape = B,L,L

        probs =  F.softmax(scores, dim=-1) # B,L,L
        E  = torch.matmul(probs, v) # shape = B,L,E

        return E

    def forward(self, inputs,debug=False):

        X_emb = self.embeddings(inputs)

        self.posicional_encoder(X_emb)

        residual = X_emb

        q = self.W_q(X_emb).to(device) #Shape = (B,L,D)
        k = self.W_k(X_emb).to(device) #Shape = (B,L,D)
        v = self.W_v(X_emb).to(device) #Shape = (B,L,D)

        X_emb = self.Attention(q=q, k=k, v=v)

        X_emb = self.W_o(X_emb)

        X_emb += residual
        
        hidden = torch.relu(self.layer1(X_emb.reshape(len(inputs),-1)))
        logits = torch.relu(self.layer2(hidden))

        logits = self.bn1(logits)

        logits = self.dropout(logits)

        logits = self.layer3(logits)

        return logits

In [47]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)


load_dict = torch.load("gdrive/MyDrive/Colab Notebooks/"+"best_model_25_may.pt")
model.load_state_dict(load_dict['model_state_dict'])
model.to(device)
model.load_state_dict

<bound method Module.load_state_dict of LanguageModel(
  (embeddings): Embedding(29794, 128)
  (posicional_encoder): PositionalEncoder()
  (W_q): Linear(in_features=128, out_features=128, bias=False)
  (W_k): Linear(in_features=128, out_features=128, bias=False)
  (W_v): Linear(in_features=128, out_features=128, bias=False)
  (W_o): Linear(in_features=128, out_features=128, bias=False)
  (layer1): Linear(in_features=1152, out_features=256, bias=True)
  (layer2): Linear(in_features=256, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (layer3): Linear(in_features=512, out_features=29794, bias=True)
)>

In [ ]:
max_examples = 1000_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)


load_dict = torch.load("gdrive/MyDrive/Colab Notebooks/"+"best_model_25_may.pt")
model.load_state_dict(load_dict['model_state_dict'])
model.to(device)


train_loader = DataLoader(training_dataset, batch_size=2048, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=2048)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()

save_best_model = SaveBestModel()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))
            
            last_loss = valid_ppl

            save_best_model(
              last_loss, 0, model, optimizer, nn.functional.cross_entropy
              )

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

Best validation loss: 113.75435092966318
0 steps; 0 examples so far; train ppl: 12567.00, valid ppl: 113.75


In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()

load_dict = torch.load("gdrive/MyDrive/Colab Notebooks/"+"best_model_26_may.pt")
model.load_state_dict(load_dict['model_state_dict'])
model.to(device)

test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz muito
Eu gosto de comer pizza pois me faz muito bem
Eu gosto de comer pizza pois me faz muito bem,
Eu gosto de comer pizza pois me faz muito bem, mas
Eu gosto de comer pizza pois me faz muito bem, mas não
Eu gosto de comer pizza pois me faz muito bem, mas não é
Eu gosto de comer pizza pois me faz muito bem, mas não é o
Eu gosto de comer pizza pois me faz muito bem, mas não é o que
Eu gosto de comer pizza pois me faz muito bem, mas não é o que você
Eu gosto de comer pizza pois me faz muito bem, mas não é o que você vai
Eu gosto de comer pizza pois me faz muito bem, mas não é o que você vai fazer
Eu gosto de comer pizza pois me faz muito bem, mas não é o que você vai fazer?
Eu gosto de comer pizza pois me faz muito bem, mas não é o que você vai fazer??
Eu gosto de comer pizza pois me faz muito bem, mas não é o que você vai fazer???
Eu gosto de comer pizza pois me faz muito bem, mas não é o que você vai fazer????
Eu gosto de comer pizza pois me faz muit

In [ ]:
prompt = 'Correr é interessante quando nos queremos esvaziar a'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Correr é interessante quando nos queremos esvaziar a vida
Correr é interessante quando nos queremos esvaziar a vida de
Correr é interessante quando nos queremos esvaziar a vida de um
Correr é interessante quando nos queremos esvaziar a vida de um homem
Correr é interessante quando nos queremos esvaziar a vida de um homem que
Correr é interessante quando nos queremos esvaziar a vida de um homem que se
Correr é interessante quando nos queremos esvaziar a vida de um homem que se encontra
Correr é interessante quando nos queremos esvaziar a vida de um homem que se encontra em
Correr é interessante quando nos queremos esvaziar a vida de um homem que se encontra em um
Correr é interessante quando nos queremos esvaziar a vida de um homem que se encontra em um lugar
Correr é interessante quando nos queremos esvaziar a vida de um homem que se encontra em um lugar de
Correr é interessante quando nos queremos esvaziar a vida de um homem que se encontra em um lugar de um
Correr é interessante quan

In [ ]:
prompt = 'sao paulo é uma cidade grande com muitas coisas'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

sao paulo é uma cidade grande com muitas coisas que
sao paulo é uma cidade grande com muitas coisas que não
sao paulo é uma cidade grande com muitas coisas que não se
sao paulo é uma cidade grande com muitas coisas que não se preocupa
sao paulo é uma cidade grande com muitas coisas que não se preocupam
sao paulo é uma cidade grande com muitas coisas que não se preocupam com
sao paulo é uma cidade grande com muitas coisas que não se preocupam com a
sao paulo é uma cidade grande com muitas coisas que não se preocupam com a vida
sao paulo é uma cidade grande com muitas coisas que não se preocupam com a vida.
sao paulo é uma cidade grande com muitas coisas que não se preocupam com a vida. A
sao paulo é uma cidade grande com muitas coisas que não se preocupam com a vida. A gente
sao paulo é uma cidade grande com muitas coisas que não se preocupam com a vida. A gente não
sao paulo é uma cidade grande com muitas coisas que não se preocupam com a vida. A gente não é
sao paulo é uma cidade gran

In [ ]:
prompt = 'morar na capital, tras algumas facilidades'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

morar na capital, tras algumas facilidades de
morar na capital, tras algumas facilidades de uma
morar na capital, tras algumas facilidades de uma forma
morar na capital, tras algumas facilidades de uma forma de
morar na capital, tras algumas facilidades de uma forma de entretenimento
morar na capital, tras algumas facilidades de uma forma de entretenimento,
morar na capital, tras algumas facilidades de uma forma de entretenimento, mas
morar na capital, tras algumas facilidades de uma forma de entretenimento, mas não
morar na capital, tras algumas facilidades de uma forma de entretenimento, mas não é
morar na capital, tras algumas facilidades de uma forma de entretenimento, mas não é uma
morar na capital, tras algumas facilidades de uma forma de entretenimento, mas não é uma boa
morar na capital, tras algumas facilidades de uma forma de entretenimento, mas não é uma boa opção
morar na capital, tras algumas facilidades de uma forma de entretenimento, mas não é uma boa opção.
morar na cap

## Re-train third time